In [ ]:

import numpy as np
import pydicom
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from scipy.ndimage.interpolation import rotate
import projd

%matplotlib inline
sns.set()

In [ ]:
# luna data dir

# iterate over patients in a data set


In [ ]:
#Example pipeline working with only one 'patient'
proj_dir = projd.cwd_token_dir('.git') # project root dir
print(proj_dir)
data_path = str(Path(proj_dir, 'data/DICOM/EXP00000'))
print(data_path)
output_path = working_path = str(Path(proj_dir, 'data'))
g = glob(data_path + '/*')

# Print out the first 5 file names to verify we're in the right folder.
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g))
print ('\n'.join(g[:5]))

In [ ]:
#      
# Loop over the image files and store everything into a list.
# 

def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
        
    return slices

def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    
    #Images already in int16, but just to confirm~ note only 12 bits used
    image = image.astype(np.int16)

    #If anything outside of scan set to 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

id=0
patient = load_scan(data_path)
imgs = get_pixels_hu(patient)

In [ ]:
print(imgs.shape)

In [ ]:
#Plot histogram for the data
plt.hist(imgs.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
#See some examples
def sample_stack(stack, rows=3, cols=3, start_with=0, show_every=3, r=0):
    fig,ax = plt.subplots(rows,cols,figsize=[20,20])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        
        if r == 0:
            ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        else:
            ax[int(i/rows),int(i % rows)].imshow(rotate(stack[ind], r),cmap='gray')
        
        
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

sample_stack(imgs, show_every=10)

Since the stack contains 50% overlap between every image, let's split it into two different stacks, just by even and odd. This leaves us with two disjoint 3D images. 

In [ ]:
even_img = np.zeros((107,512,512))
odd_img = np.zeros((107,512,512)) #One every in odd, only 213 images, just set to 0

n1 = 0
n2 = 0

for z in range(len(imgs)):
    if z % 2 == 0:
        even_img[n1] = imgs[z]
        n1+=1
    else:
        odd_img[n2] = imgs[z]
        n2+=1

In [ ]:
sample_stack(even_img, start_with=40, show_every=1)

In [ ]:
sample_stack(odd_img, start_with=40, show_every=1)

This works fine for the axial view (the default view as shown above), but if we want to view images from the sagittal or the coronal views- we need to resample.  

In [ ]:
#Find the dimensions of each pixel
pixel_dim = patient[0].PixelSpacing
print("Pixel Dimensions: ", pixel_dim)

slice_thickness = patient[0].SliceThickness
print("Slice Thickness: ", slice_thickness)

I think it makes sense to just resample the slice thickness down to the same as the pixel dimensions, since they are already uniform.

In [ ]:
def resample(image, scan, new_spacing):
    
    # Determine current pixel spacing
    #spacing = map(float, ([scan[0].SliceThickness] + scan[0].PixelSpacing))
    #spacing = np.array(list(spacing))
    
    spacing = np.array([.9, 0.29296875, 0.29296875])

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    
    new_spacing = spacing / real_resize_factor
    
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    return image, new_spacing

# Resample to 1mm spacing, typical for deep learning on CT examples seen so far.
# This might make a good hyperparam.  We might need fine detail that this size change blurs.

print('Shape before resampleing\t', even_img.shape)
post_even, even_spacing = resample(even_img, patient, [1,1,1])
print ("Shape after resampling\t", post_even.shape, even_spacing)

print('Shape before resampleing\t', odd_img.shape)
post_odd, odd_spacing = resample(odd_img, patient, [1,1,1])
print ("Shape after resampling\t", post_odd.shape, odd_spacing)

Now if we look at the images post sampling, they look simmilar, but now each slice represents ~.3mm instead of .9mm.

In [ ]:
sample_stack(post_even, start_with=40, show_every=1)

In [ ]:
sample_stack(post_odd, start_with=40, show_every=5)

It is also easy to now view these images from new views (let's just look at the even for simplicity)

First, coronal~

In [ ]:
#Coronal transposition
p = post_even.transpose(1,0,2)

print("New shape:", p.shape)

sample_stack(p, start_with=30, show_every=3, r=0)

In [ ]:
p = post_even.transpose(2,0,1)

print("New shape:", p.shape)
sample_stack(p, start_with=30, show_every=10)

Note, I'm not quite sure on the standard, the sagittal seems to be in the standard orientation, but the coronal might be mirrored? Not that it matters too much anyway, I expect mirroring/random skews will be applied regardless. 